In [ ]:
%pip install turicreate

In [6]:
import json
import pandas as pd
import glob
import os
import turicreate as tc
import shutil
import numpy as np

#os.remove('/Users/marcoincerti/Desktop/dataML/Data_ready.csv')

shutil.rmtree(r'/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/dataML/train')
shutil.rmtree(r'/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/dataML/test')

In [7]:
def normalize_data(data):
    prev_relative = data[0]['altitude']['relativeAltitude']
    data[0]['altitude']['relativeAltitude'] = 0.0
    
    prev_pressure = data[0]['altitude']['pressure']
    data[0]['altitude']['pressure'] = 0.0
    
    for d in range(1, len(data)):
        tmp = data[d]['altitude']['relativeAltitude']
        tmp_pressure = data[d]['altitude']['pressure']
        if tmp - prev_relative == 0:
            data[d]['altitude']['relativeAltitude'] = data[d -1]['altitude']['relativeAltitude']
        else:
            data[d]['altitude']['relativeAltitude'] = float(tmp - prev_relative)
        if tmp - prev_relative == 0:
            data[d]['altitude']['pressure'] = data[d -1]['altitude']['pressure']
        else:
            data[d]['altitude']['pressure'] = float(tmp_pressure - prev_pressure)
        prev_relative = tmp
        prev_pressure = tmp_pressure
    
    return data

In [10]:
i = 0
df_list = []
# Opening JSON file
for curr_file in glob.glob("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/Dataset/50Hz/stay/*s.json"):
    # Opening JSON file
  with open(curr_file) as json_file:
    data = json.load(json_file)

    data = data['motionRecords']
    tmp = data
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] != 0:
       data.pop(0)
    
    if len(data) > 10:
        data_norm = normalize_data(data)
        df = pd.json_normalize(data_norm, sep='_') 
    
        df['sessionId'] = i
        i +=1
        df['activity'] = "stay"
        df_list.append(df)
 
i = 0       
for curr_file in glob.glob("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/Dataset/50Hz/climb/*s.json"):
    # Opening JSON file
  with open(curr_file) as json_file:
    data = json.load(json_file)
    
    data = data['motionRecords']
    tmp = data
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] != 0:
       data.pop(0)
    
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] == 0:
       data.pop(0)
       
    if len(data) > 10:
        data_norm = normalize_data(data)
        df = pd.json_normalize(data_norm, sep='_') 
    
        df['sessionId'] = i
        i +=1
        df['activity'] = "climb"
        df_list.append(df)
    
i = 0
# Opening JSON file
for curr_file in glob.glob("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/Dataset/50Hz/descend/*s.json"):
    # Opening JSON file
  with open(curr_file) as json_file:
    data = json.load(json_file)
    
    data = data['motionRecords']
    
    tmp = data
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] != 0:
       data.pop(0)
       
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] == 0:
       data.pop(0)
        
    if len(data) > 10:
        data_norm = normalize_data(data)
        df = pd.json_normalize(data_norm, sep='_') 
    
        df['sessionId'] = i
        i +=1
        df['activity'] = "descend"
        df_list.append(df)
    


i = 0
# Opening JSON file
for curr_file in glob.glob("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/Dataset/50Hz/elevator_up/*s.json"):
    # Opening JSON file
  with open(curr_file) as json_file:
    data = json.load(json_file)
 
    # for reading nested data [0] represents
    # the index value of the list
    data = data['motionRecords']
    
    tmp = data
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] != 0:
       data.pop(0)
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] == 0:
       data.pop(0)
        
    if len(data) > 10:
        data_norm = normalize_data(data)
        df = pd.json_normalize(data_norm, sep='_') 
    
        df['sessionId'] = i
        i +=1
        df['activity'] = "elevator_up"
        df_list.append(df)
    

i = 0
# Opening JSON file
for curr_file in glob.glob("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/Dataset/50Hz/elevator_down/*s.json"):
    # Opening JSON file
  with open(curr_file) as json_file:
    data = json.load(json_file)
 
    # for reading nested data [0] represents
    # the index value of the list
    data = data['motionRecords']
    
    tmp = data
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] != 0:
       data.pop(0)
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] == 0:
       data.pop(0)
        
    if len(data) > 10:
        data_norm = normalize_data(data)
        df = pd.json_normalize(data_norm, sep='_') 
    
        df['sessionId'] = i
        i +=1
        df['activity'] = "eleveator_down"
        df_list.append(df)
    

df_final = pd.concat(df_list)
print(df_final.count())
df_final = df_final.dropna()
#df_final['altitude_relativeAltitude'].mask(df_final['altitude_relativeAltitude'] < 0.001, 0, inplace=True)
df_final['altitude_relativeAltitude'] = df_final['altitude_relativeAltitude'].replace([0],float(0.001))
df_final['altitude_pressure'] = df_final['altitude_pressure'].replace([0],float(0.0001))
#df_final.drop(df_final[df_final.altitude_relativeAltitude == 0].index, inplace=True)
print(df_final.count())
#df_final.head()

recordTimestamp                 99215
attitude_roll                   99215
attitude_pitch                  99215
attitude_yaw                    99215
altitude_pressure               99215
altitude_relativeAltitude       99215
rotationRate_x                  99215
rotationRate_y                  99215
rotationRate_z                  99215
accelerationWithOutGravity_x    99215
accelerationWithOutGravity_y    99215
accelerationWithOutGravity_z    99215
sessionId                       99215
activity                        99215
dtype: int64
recordTimestamp                 99215
attitude_roll                   99215
attitude_pitch                  99215
attitude_yaw                    99215
altitude_pressure               99215
altitude_relativeAltitude       99215
rotationRate_x                  99215
rotationRate_y                  99215
rotationRate_z                  99215
accelerationWithOutGravity_x    99215
accelerationWithOutGravity_y    99215
accelerationWithOutGravity_z    99215

In [5]:
# Column name mappings
cols = {
    #"recordTimestamp": "timestamp",
    #"attitude_roll": "attitude_roll",
    #"attitude_pitch": "attitude_pitch",
    #"attitude_yaw": "attitude_yaw",
    "altitude_pressure": "altitude_pressure",
    "altitude_relativeAltitude": "relativeAltitude",
    "rotationRate_x": "rotationRate_x",
    "rotationRate_y": "rotationRate_y",
    "rotationRate_z": "rotationRate_z",
    "accelerationWithOutGravity_x": "acceleration_x",
    "accelerationWithOutGravity_y": "acceleration_y",
    "accelerationWithOutGravity_z": "acceleration_z",
    "sessionId": "session_id",
    "activity": "activity"
    }

csv_cols = ["rotationRate_x", "rotationRate_y", "rotationRate_z", "acceleration_x", "acceleration_y",
            "acceleration_z","relativeAltitude", "altitude_pressure" ]

# Load csv data and rename columns
sf = tc.SFrame(data=df_final[list(cols.keys())]).rename(cols)

# Remove missing activies
sf = sf[sf['activity'] != '']
acts = sf['activity'].unique()

# Split data into training and testing sets
#train, test = sf.random_split(0.8)
train, test = tc.activity_classifier.util.random_split_by_session(sf, session_id='session_id', fraction=0.8)

# Write out training data
path = "train/"
os.mkdir(path)
for a in acts:
  # Check for folder
  cls_path = path + a + "/"
  if not os.path.exists(cls_path):
    os.mkdir(cls_path)
# Split data by activity & session and write to file
  sf_act = train[train['activity'] == a]
  for s in sf_act['session_id'].unique():
    sf_act[sf_act['session_id'] == s][csv_cols].save(cls_path + str(a)+ "_" + str(s) + ".csv")
# Write out testing data
path = "test/"
os.mkdir(path)
for a in acts:
  # Check for folder
  cls_path = path + a + "/"
  if not os.path.exists(cls_path):
    os.mkdir(cls_path)
# Split data by activity & session and write to file
  sf_act = test[test['activity'] == a]
  for s in sf_act['session_id'].unique():
    sf_act[sf_act['session_id'] == s][csv_cols].save(cls_path + str(a) + "_" + str(s) + ".csv")
